# Floating potential approach

The currents are imposed and the floating potential is determined.
In addition, one electrode is set to ground.

An efficient block preconditioner needs to be found to solve this kind of system
with an iterative solver.

In [ ]:
import netgen.occ as occ
import ngsolve
from netgen.webgui import Draw as DrawGeo
from ngsolve.webgui import Draw

In [ ]:
unit_square = occ.unit_square_shape

In [ ]:
e1 = occ.Circle((0.2, 0.2), r=0.1)
e2 = occ.Circle((0.5, 0.5), r=0.1)
e3 = occ.Circle((0.8, 0.8), r=0.1)

In [ ]:
electrodes = [e1.Face(), e2.Face(), e3.Face()]
for idx, electrode in enumerate(electrodes):
    for edge in electrode.edges:
        edge.name = f"Electrode_{idx}"
    unit_square = unit_square - electrode

In [ ]:
DrawGeo(unit_square)

In [ ]:
mesh = ngsolve.Mesh(occ.OCCGeometry(unit_square, dim=2).GenerateMesh())
mesh.Curve(2)

In [ ]:
Draw(mesh)
print(mesh.GetBoundaries())

In [ ]:
V = ngsolve.H1(mesh, order=2, dirichlet="Electrode_2")
lm = ngsolve.SurfaceL2(mesh, order=1, dirichlet="bottom|left|right|top|Electrode_2")
V_fixed = []
for _ in range(2):
    V_fixed.append(ngsolve.NumberSpace(mesh, order=0))
fes = ngsolve.FESpace([V, lm, *V_fixed])

print("DOFs:")
print(V.ndof)
print(lm.ndof)
for V_fix in V_fixed:
    print(V_fix.ndof)

I_1 = -0.5  # A
I_2 = -0.5
I_3 = 1.0
I_all = [I_1, I_2, I_3]

trial = fes.TrialFunction()
test = fes.TestFunction()
u, lam = trial[:2]
v, mu = test[:2]

a = ngsolve.BilinearForm(fes)
a += ngsolve.grad(u) * ngsolve.grad(v) * ngsolve.dx

boundaries = [f"Electrode_{idx}" for idx in range(2)]
for ufix, vfix, boundary in zip(trial[2:], test[2:], boundaries, strict=False):
    a += (u * mu + v * lam) * ngsolve.ds(boundary)
    a += -(ufix * mu + vfix * lam) * ngsolve.ds(boundary)

f = ngsolve.LinearForm(fes)
f += 0.0 * v * ngsolve.dx
for I_x, vfix, boundary in zip(I_all, test[2:], boundaries, strict=False):
    length = ngsolve.Integrate(
        ngsolve.CoefficientFunction(1.0) * ngsolve.ds(boundary), mesh
    )
    f += I_x / length * vfix * ngsolve.ds(boundary)
length = ngsolve.Integrate(
    ngsolve.CoefficientFunction(1.0) * ngsolve.ds("Electrode_2"), mesh
)
f += I_3 / length * v * ngsolve.ds("Electrode_2")

a.Assemble()
f.Assemble()

gfu = ngsolve.GridFunction(fes)

r = f.vec.CreateVector()
r.data = f.vec - a.mat * gfu.vec
gfu.vec.data += a.mat.Inverse(freedofs=fes.FreeDofs()) * r
Draw(gfu.components[0])

# Check output

Try to understand orientation of normal vector!

In [ ]:
normal_current = gfu.components[1]
normal_component = ngsolve.specialcf.normal(2)
current_density = (
    ngsolve.BoundaryFromVolumeCF(-ngsolve.grad(gfu.components[0])) * normal_component
)
print(
    ngsolve.Integrate(normal_current * ngsolve.ds("Electrode_0"), mesh),
    ngsolve.Integrate(current_density * ngsolve.ds("Electrode_0"), mesh),
)
print(
    ngsolve.Integrate(normal_current * ngsolve.ds("Electrode_1"), mesh),
    ngsolve.Integrate(current_density * ngsolve.ds("Electrode_1"), mesh),
)
print(
    ngsolve.Integrate(normal_current * ngsolve.ds("Electrode_2"), mesh),
    ngsolve.Integrate(current_density * ngsolve.ds("Electrode_2"), mesh),
)

# Floating potentials

In [ ]:
print(gfu.components[2].vec)
print(gfu.components[3].vec)

# Field plot

In [ ]:
Draw(-ngsolve.grad(gfu.components[0]), mesh)